In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

outlets = pd.read_csv('outlets.csv')
items = pd.read_csv('items.csv')
ireplies = pd.read_csv('ireplies.csv')
metadata = pd.read_csv('metadata.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df1 = pd.merge(outlets, items, on = 'newsoutlet_id', how='inner')
df2 = pd.merge(df1,metadata, on = 'news_id', how='inner')

In [3]:
m = pd.merge(df2, ireplies, left_on='news_id', right_on='in_reply_to_news_id', how='inner')

In [4]:
# comentarios toxicos (0 - não tóxico; 1 - tóxico)

m['Toxic'] = np.where(m['perspective_toxicity']<0.5, 0, 1)

In [5]:
m.Toxic.value_counts()

0    719403
1    242542
Name: Toxic, dtype: int64

In [6]:
# Percentagem comentários tóxicos (0 - não tóxico; 1 - tóxico)
m['Toxic'].value_counts(normalize=True) * 100

0    74.786292
1    25.213708
Name: Toxic, dtype: float64

In [7]:
# Para cada notícia a soma, contagem e percentagem de comentários tóxicos
tSum = m.groupby('news_id')['Toxic'].sum()
tCount = m.groupby('news_id')['Toxic'].count()
tPerc = m.groupby('news_id')['Toxic'].mean()*100

news_stat = pd.DataFrame({'news_id':tSum.index, 'Toxic sum':tSum, 'Toxic count':tCount, 'Toxic percentage': tPerc})
news_stat

,news_id,Toxic sum,Toxic count,Toxic percentage
news_id,,,,
1078283917315186688,1078283917315186688,13,79,16.455696
1078284292072132609,1078284292072132609,3,8,37.500000
1078284432530903041,1078284432530903041,0,21,0.000000
1078284510276534273,1078284510276534273,5,31,16.129032
1078284562466308096,1078284562466308096,4,22,18.181818
...,...,...,...,...
1084620036956770306,1084620036956770306,1,11,9.090909
1084620301877551105,1084620301877551105,1,19,5.263158
1084621455168626688,1084621455168626688,0,7,0.000000


### Métrica 1
São consideradas tóxicas as notícias em que a percentagem dos seus comentários tóxicos sejam maiores que a percentagem média.

In [8]:
# média da percentagem de comentários tóxicos de cada notícia
media = news_stat['Toxic percentage'].mean()
media

21.34020249677203

In [9]:
# notícias consideradas geradoras de toxicidade (pela MÉDIA) 
toxic_mean = news_stat[news_stat['Toxic percentage'] > media]
toxic_mean

,news_id,Toxic sum,Toxic count,Toxic percentage
news_id,,,,
1078284292072132609,1078284292072132609,3,8,37.500000
1078284953849348096,1078284953849348096,6,14,42.857143
1078285650317766656,1078285650317766656,8,31,25.806452
1078287429155676161,1078287429155676161,2,6,33.333333
1078288893018017792,1078288893018017792,3,10,30.000000
...,...,...,...,...
1084616110547324929,1084616110547324929,28,92,30.434783
1084616466325012480,1084616466325012480,16,16,100.000000
1084616587641004037,1084616587641004037,18,58,31.034483


In [10]:
# news_ids de notícias geradoras de toxicidade (lista)
txc_ids_m = list(toxic_mean['news_id'])

# Notícias classificadas
items_m1 = df2
items_m1['Toxic_Class'] = np.where(items_m1['news_id'].isin(txc_ids_m), 1, 0)

# Seleção de colunas
news_M1 = items_m1[['news_id','newsoutlet_id','new_title','news_text','Toxic_Class']]
news_M1

,news_id,newsoutlet_id,new_title,news_text,Toxic_Class
0,1080369723227979776,428333,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0
1,1078295012532736001,428333,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0
2,1079909388524154880,428333,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0
3,1081296207790538753,428333,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...,0
4,1081777463351541762,428333,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...,1
...,...,...,...,...,...
13006,1084477695982030848,16973333,Paris explosion: Body found in rubble of baker...,Rescue workers have discovered a body in the r...,1
13007,1080878204620021760,16973333,China to launch self-driving bullet trains tha...,China will introduce the world’s first driverl...,0
13008,1082585338898513921,16973333,Yellow vest protests: Who are the 'far-right e...,Police have been accused of ignoring the haras...,0
13009,1078712411539034112,16973333,Italy plans to stop arms sales to Saudi Arabia...,Italian Prime Minister Giuseppe Conte has said...,0


In [11]:
stat1 = news_M1['Toxic_Class'].value_counts()
stat2 = news_M1['Toxic_Class'].value_counts(normalize=True)*100

print('# Métrica 1 (Média) #')
print(f'{stat1[1]} ({stat2[1]}%) Notícias Tóxicas')
print(f'{stat1[0]} ({stat2[0]}%) Notícias Não Tóxicas')

# Métrica 1 (Média) #
4829 (37.11474905848897%) Notícias Tóxicas
8182 (62.88525094151103%) Notícias Não Tóxicas


In [12]:
s = news_M1['Toxic_Class']
counts = s.value_counts()
percentage = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'counts': counts, 'percentage': percentage})

,counts,percentage
0,8182,62.9%
1,4829,37.1%


### Métrica 2
São consideradas tóxicas as notícias em que a percentagem dos seus comentários tóxicos sejam maiores que a percentagem mediana.

In [13]:
# mediana da percentagem de comentários tóxicos de cada notícia
mediana = news_stat['Toxic percentage'].median()
mediana

16.666666666666664

In [14]:
# notícias consideradas geradoras de toxicidade (pela MEDIANA) 
toxic_median = news_stat[news_stat['Toxic percentage'] > mediana]
toxic_median

,news_id,Toxic sum,Toxic count,Toxic percentage
news_id,,,,
1078284292072132609,1078284292072132609,3,8,37.500000
1078284562466308096,1078284562466308096,4,22,18.181818
1078284953849348096,1078284953849348096,6,14,42.857143
1078285087236591616,1078285087236591616,9,43,20.930233
1078285650317766656,1078285650317766656,8,31,25.806452
...,...,...,...,...
1084616587641004037,1084616587641004037,18,58,31.034483
1084616634814345217,1084616634814345217,6,17,35.294118
1084619901208272896,1084619901208272896,97,203,47.783251


In [15]:
# news_ids de notícias geradoras de toxicidade (lista)
txc_ids_md = list(toxic_median['news_id'])

# Notícias classificadas
items_m2 = df2
items_m2['Toxic_Class'] = np.where(items_m2['news_id'].isin(txc_ids_md), 1, 0)

# Seleção de colunas
news_M2 = items_m2[['news_id','newsoutlet_id','new_title','news_text','Toxic_Class']]
news_M2

,news_id,newsoutlet_id,new_title,news_text,Toxic_Class
0,1080369723227979776,428333,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0
1,1078295012532736001,428333,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0
2,1079909388524154880,428333,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0
3,1081296207790538753,428333,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...,0
4,1081777463351541762,428333,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...,1
...,...,...,...,...,...
13006,1084477695982030848,16973333,Paris explosion: Body found in rubble of baker...,Rescue workers have discovered a body in the r...,1
13007,1080878204620021760,16973333,China to launch self-driving bullet trains tha...,China will introduce the world’s first driverl...,0
13008,1082585338898513921,16973333,Yellow vest protests: Who are the 'far-right e...,Police have been accused of ignoring the haras...,0
13009,1078712411539034112,16973333,Italy plans to stop arms sales to Saudi Arabia...,Italian Prime Minister Giuseppe Conte has said...,0


In [16]:
stat1 = news_M2['Toxic_Class'].value_counts()
stat2 = news_M2['Toxic_Class'].value_counts(normalize=True)*100

print('# Métrica 2 (Mediana) #')
print(f'{stat1[1]} ({stat2[1]}%) Notícias Tóxicas')
print(f'{stat1[0]} ({stat2[0]}%) Notícias Não Tóxicas')

# Métrica 2 (Mediana) #
6376 (49.00468834063485%) Notícias Tóxicas
6635 (50.99531165936515%) Notícias Não Tóxicas


In [17]:
s = news_M2['Toxic_Class']
counts = s.value_counts()
percentage = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'counts': counts, 'percentage': percentage})

,counts,percentage
0,6635,51.0%
1,6376,49.0%


### Métrica 3

In [18]:
news_stat['Toxic percentage'].describe()

count    13011.000000
mean        21.340202
std         20.642094
min          0.000000
25%         10.000000
50%         16.666667
75%         26.666667
max        100.000000
Name: Toxic percentage, dtype: float64

In [19]:
thirdQ = np.percentile(news_stat['Toxic percentage'], 75)
thirdQ

26.666666666666668

In [20]:
# notícias consideradas geradoras de toxicidade (pelo 3º Quartil) 
toxic_3Q = news_stat[news_stat['Toxic percentage'] > thirdQ]
toxic_3Q

,news_id,Toxic sum,Toxic count,Toxic percentage
news_id,,,,
1078284292072132609,1078284292072132609,3,8,37.500000
1078284953849348096,1078284953849348096,6,14,42.857143
1078287429155676161,1078287429155676161,2,6,33.333333
1078288893018017792,1078288893018017792,3,10,30.000000
1078289459039358978,1078289459039358978,22,22,100.000000
...,...,...,...,...
1084616110547324929,1084616110547324929,28,92,30.434783
1084616466325012480,1084616466325012480,16,16,100.000000
1084616587641004037,1084616587641004037,18,58,31.034483


In [21]:
# news_ids de notícias geradoras de toxicidade (lista)
txc_ids_3Q = list(toxic_3Q['news_id'])

# Notícias classificadas
items_m3 = df2
items_m3['Toxic_Class'] = np.where(items_m3['news_id'].isin(txc_ids_3Q), 1, 0)

# Seleção de colunas
news_M3 = items_m3[['news_id','newsoutlet_id','new_title','news_text','Toxic_Class']]
news_M3

,news_id,newsoutlet_id,new_title,news_text,Toxic_Class
0,1080369723227979776,428333,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",0
1,1078295012532736001,428333,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0
2,1079909388524154880,428333,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,0
3,1081296207790538753,428333,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...,0
4,1081777463351541762,428333,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...,1
...,...,...,...,...,...
13006,1084477695982030848,16973333,Paris explosion: Body found in rubble of baker...,Rescue workers have discovered a body in the r...,0
13007,1080878204620021760,16973333,China to launch self-driving bullet trains tha...,China will introduce the world’s first driverl...,0
13008,1082585338898513921,16973333,Yellow vest protests: Who are the 'far-right e...,Police have been accused of ignoring the haras...,0
13009,1078712411539034112,16973333,Italy plans to stop arms sales to Saudi Arabia...,Italian Prime Minister Giuseppe Conte has said...,0


In [22]:
s = news_M3['Toxic_Class']
counts = s.value_counts()
percentage = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'counts': counts, 'percentage': percentage})

,counts,percentage
0,9812,75.4%
1,3199,24.6%


# Métrica 4

In [23]:
firstQ = np.percentile(news_stat['Toxic percentage'], 25)
firstQ

10.0

In [24]:
# notícias consideradas geradoras de toxicidade (pelo 1º Quartil) 
toxic_1Q = news_stat[news_stat['Toxic percentage'] > firstQ]
toxic_1Q

,news_id,Toxic sum,Toxic count,Toxic percentage
news_id,,,,
1078283917315186688,1078283917315186688,13,79,16.455696
1078284292072132609,1078284292072132609,3,8,37.500000
1078284510276534273,1078284510276534273,5,31,16.129032
1078284562466308096,1078284562466308096,4,22,18.181818
1078284953849348096,1078284953849348096,6,14,42.857143
...,...,...,...,...
1084616634814345217,1084616634814345217,6,17,35.294118
1084619901208272896,1084619901208272896,97,203,47.783251
1084619909076828160,1084619909076828160,7,39,17.948718


In [25]:
# news_ids de notícias geradoras de toxicidade (lista)
txc_ids_1Q = list(toxic_1Q['news_id'])

# Notícias classificadas
items_m4 = df2
items_m4['Toxic_Class'] = np.where(items_m4['news_id'].isin(txc_ids_1Q), 1, 0)

# Seleção de colunas
news_M4 = items_m4[['news_id','newsoutlet_id','new_title','news_text','Toxic_Class']]
news_M4

,news_id,newsoutlet_id,new_title,news_text,Toxic_Class
0,1080369723227979776,428333,Winning ticket sold for $425 million New Year'...,"(CNN) Another year, another lottery you didn't...",1
1,1078295012532736001,428333,Clashes in Tunisia after journalist sets himse...,(CNN) Clashes in Tunisia have continued betwee...,0
2,1079909388524154880,428333,"Chief Justice Roberts avoids controversy, prai...",(CNN) — Chief Justice John Roberts steered cle...,1
3,1081296207790538753,428333,Mueller grand jury gets 6 month extension,Washington (CNN) Special counsel Robert Muelle...,1
4,1081777463351541762,428333,Jazmine Barnes shooting: Police interview 'per...,(CNN) Detectives investigating the drive-by mu...,1
...,...,...,...,...,...
13006,1084477695982030848,16973333,Paris explosion: Body found in rubble of baker...,Rescue workers have discovered a body in the r...,1
13007,1080878204620021760,16973333,China to launch self-driving bullet trains tha...,China will introduce the world’s first driverl...,1
13008,1082585338898513921,16973333,Yellow vest protests: Who are the 'far-right e...,Police have been accused of ignoring the haras...,1
13009,1078712411539034112,16973333,Italy plans to stop arms sales to Saudi Arabia...,Italian Prime Minister Giuseppe Conte has said...,1


In [26]:
s = news_M4['Toxic_Class']
counts = s.value_counts()
percentage = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.DataFrame({'counts': counts, 'percentage': percentage})

,counts,percentage
1,9600,73.8%
0,3411,26.2%


In [27]:
# export do csv
news_M1.to_csv('news_M1.csv', index=False)
news_M2.to_csv('news_M2.csv', index=False)
news_M3.to_csv('news_M3.csv', index=False)
news_M4.to_csv('news_M4.csv', index=False)